# Hyperparameter Tunning

Experiment setup and the HParams experiment summary

Experiment with  hyperparameters in the model:

1. Number of units in the first dense layer
2. Dropout rate in the dropout layer
3. Optimizer

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn import datasets, metrics, model_selection


In [ ]:
# clear previous log
# rm -rvf logs

# Clear any logs from previous runs
# rm -rf 'logs/ANN/fit/'

# rm -rvf logdir logs/ANN/fit/

# Clear any logs from previous runs
rm -rf ./logs/

In [2]:
# read data

xlsx = pd.ExcelFile('./Data/Outliers_out.xlsx')
df = pd.read_excel(xlsx, '1x 5s')
dataset= df.copy()

# Split the data into train and test

# the 'sample' method takes a certain fraction randomly
#Note that we use `random_state` to ensure the reproducibility of the examples.
train_dataset = dataset.sample(frac=0.7, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


# split datat into input and target

train_input = train_dataset.copy()
test_input = test_dataset.copy()

train_target = train_input.pop('RHOB')
test_target = test_input.pop('RHOB')



In [12]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard



In [13]:
# logdir = "logs\ANN"


from tensorboard.plugins.hparams import api as hp

Experiment setup and the HParams experiment summary

Experiment with  hyperparameters in the model:

1. Number of units in the first dense layer
2. Dropout rate in the dropout layer
3. Optimizer

In [14]:
# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([300, 200,512]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

In [ ]:
# help(hp.HParam)

# dir(hp)

# help(    tf.keras.layers.Flatten())
# help(hparams)

In [15]:
METRIC = "mse"

with tf.summary.create_file_writer('logs/ANN/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC, display_name='mse')],
  )

In [27]:
# Adapt TensorFlow runs to log hyperparameters and metrics

def create_model(hparams):
    
#normalization
    normalizer=preprocessing.Normalization()
    normalizer.adapt(np.array(test_input))
    
    model = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu', input_shape=(5,)),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(1),
  ])
    
    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='mean_absolute_error',
      metrics=['mse'],
  )
    
    logdir= 'logs/ANN/hparam_tuning/'
    model.fit(train_input, train_target,
              epochs=10,
               callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
            
              ) # Run with a few epoch to speed things up for demo purposes
    loss, mse = model.evaluate(test_input, test_target)
    return mse





In [28]:
# For each run, log an hparams summary with the hyperparameters and final mse:

def experiment(experiment_dir, hparams):
     with tf.summary.create_file_writer(experiment_dir).as_default():
        hp.hparams(hparams)
        mse = create_model(hparams)
        tf.summary.scalar(METRIC, mse, step=1)


In [ ]:
# or use keras callbacks when traing

logdir= 'logs/ANN/hparam_tuning/'

model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)

In [29]:
experiment_no = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,}

            experiment_name = f'Experiment {experiment_no}'
            print(f'Starting Experiment: {experiment_name}')
            print({h.name: hparams[h] for h in hparams})
            experiment('logs/ANN/hparam_tuning/' + experiment_name, hparams)
            experiment_no += 1

Starting Experiment: Experiment 0
{'num_units': 8, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
109/109 [==============================] - 7s 50ms/step - loss: 1.2183 - mse: 1.5323
Epoch 2/10
109/109 [==============================] - 0s 2ms/step - loss: 1.2188 - mse: 1.5335
Epoch 3/10
109/109 [==============================] - 0s 1ms/step - loss: 1.2208 - mse: 1.5402
Epoch 4/10
109/109 [==============================] - 0s 809us/step - loss: 1.2228 - mse: 1.5453
Epoch 5/10
109/109 [==============================] - 0s 801us/step - loss: 1.2293 - mse: 1.5578
Epoch 6/10
109/109 [==============================] - 0s 979us/step - loss: 1.2211 - mse: 1.5384
Epoch 7/10
109/109 [==============================] - 0s 3ms/step - loss: 1.2205 - mse: 1.5391
Epoch 8/10
109/109 [==============================] - 0s 3ms/step - loss: 1.2124 - mse: 1.5206
Epoch 9/10
109/109 [==============================] - 0s 3ms/step - loss: 1.2228 - mse: 1.5433
Epoch 10/10
47/47 [==============================]

Epoch 3/10
109/109 [==============================] - 0s 840us/step - loss: 1.2227 - mse: 1.5421
Epoch 4/10
109/109 [==============================] - 0s 798us/step - loss: 1.2252 - mse: 1.5479
Epoch 5/10
109/109 [==============================] - 0s 1ms/step - loss: 1.2222 - mse: 1.5415
Epoch 6/10
109/109 [==============================] - 0s 1ms/step - loss: 1.2306 - mse: 1.5599
Epoch 7/10
109/109 [==============================] - 0s 1ms/step - loss: 1.2223 - mse: 1.5414
Epoch 8/10
109/109 [==============================] - 0s 831us/step - loss: 1.2158 - mse: 1.5267
Epoch 9/10
109/109 [==============================] - 0s 766us/step - loss: 1.2236 - mse: 1.5431
Epoch 10/10
47/47 [==============================] - 0s 2ms/step - loss: 1.2251 - mse: 1.5488
Starting Experiment: Experiment 8
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Epoch 1/10
109/109 [==============================] - 6s 49ms/step - loss: 1.2291 - mse: 1.5538
Epoch 2/10
109/109 [=============================

109/109 [==============================] - 0s 758us/step - loss: 1.2254 - mse: 1.5488
Epoch 6/10
109/109 [==============================] - 0s 2ms/step - loss: 1.2220 - mse: 1.5408
Epoch 7/10
109/109 [==============================] - 0s 2ms/step - loss: 1.2163 - mse: 1.5297A: 0s - loss: 1.2148 - mse: 1.5
Epoch 8/10
109/109 [==============================] - 0s 3ms/step - loss: 1.2216 - mse: 1.5395
Epoch 9/10
109/109 [==============================] - 0s 4ms/step - loss: 1.2287 - mse: 1.5549
Epoch 10/10
47/47 [==============================] - 0s 737us/step - loss: 1.2251 - mse: 1.5488
Starting Experiment: Experiment 15
{'num_units': 32, 'dropout': 0.5, 'optimizer': 'adam'}
Epoch 1/10
109/109 [==============================] - 7s 52ms/step - loss: 1.2188 - mse: 1.5336
Epoch 2/10
109/109 [==============================] - 0s 2ms/step - loss: 1.2206 - mse: 1.5382
Epoch 3/10
109/109 [==============================] - 0s 2ms/step - loss: 1.2259 - mse: 1.5474
Epoch 4/10
109/109 [===========

4. Visualize the results in TensorBoard's HParams plugin

In [30]:
%tensorboard -- logdir= "logs/ANN/hparam_tuning/"

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2021-06-08 10:48:12.098025: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-06-08 10:48:12.098177: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--purge_orphaned_data BOOL] [--db URI] [--db_import]
                   [--inspect] [--version_tb] [--tag TAG] [--event_file PATH]
                   [--path_prefix PATH] [--window_title TEXT]
                   [--max_reload_threads COUNT] [--reload_interval SECONDS]
                   [--reload_task TYPE] [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data